In [83]:
import darts
from darts import TimeSeries
from darts.models import TCNModel
from darts.utils.likelihood_models import LaplaceLikelihood
from pytorch_lightning import Trainer
import pandas as pd
import numpy as np
import time
import httpx
import json
import sqlite3
import datetime as dt
import matplotlib as plt
import polars as pl

In [ ]:
# obtained from fuzzworks
invData = pd.read_csv('../invTypes.csv')

In [ ]:
def getMarketPrices(inplace=False):
    print('Fetching market prices')
    r = httpx.get(
        'https://esi.evetech.net/latest/markets/prices/?datasource=tranquility',)
    if (r.status_code == 200):
        df = pd.read_json(r.text)
        df['date'] = dt.datetime.now()
        con = sqlite3.connect('../data/data.db')
        df.to_sql(con=con, name="market_data", if_exists="append")
        con.close()
        if (inplace):
            return (df)
        # cur = con.cursor()
        # cur.execute('select * from market_data')
        # results = cur.fetchall()
        # print(results)
    else:
        print(f"Request did not succeed! {r.status_code}")
        rjson = json.loads(r.text)
        if (rjson['timeout'] != None or 0):
            time.sleep((rjson['timeout']/1000))
# getMarketPrices(inplace=True)
# adjusted_price, average_price, type_id

In [ ]:
def getItemPrices(id):
    # refresh database with new data
    getMarketPrices()
    con = sqlite3.connect('../data.db')
    # df = pd.read_sql_table(table_name='market_data',con=con,parse_dates='date')
    # cur = con.cursor()
    # cur.execute("select * from market_data")
    df = pd.read_sql('select * from market_data', con, parse_dates=['date'])
    selected_df = df.loc[df['type_id'] == id]
    if (len(selected_df) != 0):
        return (selected_df)
    else:
        print('Sorry, either this is not a valid item id or something went wrong. :P')

In [ ]:
def getIDInfo(id):
    selected_row = invData.loc[invData['typeID'] == id]
    return selected_row
# getIDInfo(17136)

In [ ]:
def getIdFromName(name):
    # NOTE: This function could be improved with a better search engine
    selected_rows = invData[invData['typeName'].str.contains(name, False)]
    selected_rows = selected_rows[['typeName', 'typeID']]
    return (selected_rows)
# getIdFromName('Superconductors')

In [ ]:
def resample_one_minute(df):
    # Make sure the dataframe has a datetime index
    df.index = pd.to_datetime(df.index)
    # Resample the dataframe on one minute intervals
    df_resampled = df.resample('1T').mean()
    return df_resampled

In [ ]:
#Fix data for forecasting
con = sqlite3.connect('../data.db')
databaseframe = pd.read_sql(
    'select * from market_data', con, parse_dates=['date'])
con.close()
fixedData = resample_one_minute(databaseframe)

In [ ]:
con = sqlite3.connect('../data.db')
databaseframe = pd.read_sql(
    'select * from market_data', con, parse_dates=['date'])
con.close()
databaseframe.to_csv('../data.csv')

In [87]:
#Adding polars usage because increasing data processing speed
def createTimeseries(type_id):
    df = pl.read_csv('../data.csv', parse_dates=True)
    df = df.select(pl.col(["adjusted_price", "date"]).filter(pl.col('type_id') == type_id))
    df = df.with_column(pl.col('adjusted_price').cast(pl.Float32))
    set_by_minute = df.groupby_dynamic('date', every='1m').agg(pl.col('adjusted_price').mean())
    print(set_by_minute.head(5))
    print(f'shape is: {set_by_minute.shape}')
    # df = pl.read_csv('../data.csv')
    # list_select_df = df.select(["id", "color"])
    
    # I think this is how collect is used, not sure though, test on dataset later...
    series = TimeSeries.from_dataframe(df=(set_by_minute.to_pandas()), time_col='date', freq='1min')
    print(series.head())
    train, val = series.split_after(.6)
    return train, val, series

In [88]:
# con = sqlite3.connect('../data.db')
# databaseframe = pd.read_sql('select * from market_data', con, parse_dates=['date'])
# con.close()
# series = TimeSeries.from_dataframe(df=fixedData, time_col='date')
train, val, series = createTimeseries(17136)
# trend, seasonal = extract_trend_and_seasonality(series)
# print(f'trend is: {trend}, seasonal is: {seasonal}')
model = TCNModel(
    input_chunk_length=24,
    output_chunk_length=12,
    random_state=42,
    likelihood=LaplaceLikelihood(),
)
trainer = Trainer(accelerator='auto')
model.fit(train, epochs=400, verbose=True, val_series=val, trainer=trainer)
probabilistic_forecast = model.predict(n=len(val), num_samples=500)

series.plot(label="actual")
probabilistic_forecast.plot(label="probabilistic forecast")
plt.legend()
plt.show()


shape: (5, 2)
┌─────────────────────┬────────────────┐
│ date                ┆ adjusted_price │
│ ---                 ┆ ---            │
│ datetime[μs]        ┆ f32            │
╞═════════════════════╪════════════════╡
│ 2022-11-29 17:30:00 ┆ 39578.34375    │
├╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┤
│ 2022-11-29 17:31:00 ┆ 39578.34375    │
├╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┤
│ 2022-11-29 17:32:00 ┆ 39578.34375    │
├╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┤
│ 2022-11-29 17:33:00 ┆ 39578.34375    │
├╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┤
│ 2022-11-29 17:35:00 ┆ 39578.34375    │
└─────────────────────┴────────────────┘
shape is: (7570, 2)
<TimeSeries (DataArray) (date: 5, component: 1, sample: 1)>
array([[[39578.344]],

       [[39578.344]],

       [[39578.344]],

       [[39578.344]],

       [[      nan]]], dtype=float32)
Coordinates:
  * date       (date) datetime64[ns] 2022-11-29T17:30:00 ... 2022-11-29T17:34:00
  * component  (component) object 'adjusted_price'
Dimensions witho

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
c:\Users\Joe\Desktop\javis-kamiya-dojo\EVE-Market-Predictor\env\lib\site-packages\pytorch_lightning\loops\utilities.py:94: PossibleUserWarning: `max_epochs` was not set. Setting it to 1000 epochs. To train without an epoch limit, set `max_epochs=-1`.
  rank_zero_warn(

  | Name          | Type              | Params
----------------------------------------------------
0 | criterion     | MSELoss           | 0     
1 | train_metrics | MetricCollection  | 0     
2 | val_metrics   | MetricCollection  | 0     
3 | dropout       | MonteCarloDropout | 0     
4 | res_blocks    | ModuleList        | 166   
----------------------------------------------------
166       Trainable params
0         Non-trainable params
166       Total params
0.001     Total estimated model params size (MB)


Sanity Checking DataLoader 0:   0%|          | 0/2 [00:00<?, ?it/s]

ValueError: Expected parameter loc (Tensor of shape (32, 24, 1)) of distribution Laplace(loc: torch.Size([32, 24, 1]), scale: torch.Size([32, 24, 1])) to satisfy the constraint Real(), but found invalid values:
tensor([[[        nan],
         [        nan],
         [        nan],
         [        nan],
         [        nan],
         [        nan],
         [        nan],
         [        nan],
         [        nan],
         [        nan],
         [        nan],
         [        nan],
         [        nan],
         [        nan],
         [        nan],
         [        nan],
         [        nan],
         [        nan],
         [        nan],
         [        nan],
         [        nan],
         [        nan],
         [        nan],
         [        nan]],

        [[-13042.0244],
         [        nan],
         [        nan],
         [        nan],
         [        nan],
         [        nan],
         [        nan],
         [        nan],
         [        n